In [4]:
from sqlalchemy import create_engine
import pandas as pd
import json
from tqdm.notebook import tqdm
import csv
from rdkit.Chem import AllChem as rdkit
import fnmatch
from collections import defaultdict
from ast import literal_eval
from pathlib import Path
import os
import seaborn as sns
import numpy as np
from  multiprocessing import Pool
from matplotlib import pyplot as plt
import itertools as it
from typing import List
import stk
from molvs import Standardizer, standardize_smiles

In [2]:
# Dropbox path (only works locally)
paper_path = (
    Path.home()
    / "Dropbox"
    / "Apps"
    / "Overleaf"
    / "Synthetic Accessibility Paper - ACS Template"
)
figure_path = paper_path / 'Figures'

In [ ]:
def get_path():
    contains = ["SAScore", "SCScore", "RFModel"]
    fgs = ["Amine", "Aldehyde"]
    for dirpath, dirname, filename in os.walk("."):
        if (
            len(dirpath.split("/")) == 5
            and any(c in dirpath for c in contains)
            and any(fg in dirpath for fg in fgs)
        ):
            yield dirpath

In [ ]:
# Obtaining the easiest one percent to synthesise.
for path in get_path():
    path = Path(path)
    print(path)
    model_name = path.parents[1].stem.lower()
    dict_ascending = {
        "Lowest": True,
        "Highest": False,
    }
    dict_func_groups = {
        "Amine": "primary amine",
        "Aldehyde": "aldehyde",
    }
    func_group = dict_func_groups[path.stem]
    # Highest and lowest selection.
    if "Highest" in str(path) or "Lowest" in str(path):
        ascending = dict_ascending[path.parents[0].stem]
        with open(str(path.joinpath("Scores.csv")), "w") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(df_unique_reaxys.columns)
            for i in range(10):
                column = (
                    df_unique_reaxys[df_unique_reaxys["func_group"] == func_group]
                    .sort_values(model_name, ascending=ascending)
                    .iloc[i]
                )
                smiles = column["smiles"]
                mol = rdkit.MolFromSmiles(smiles)
                rdkit.EmbedMolecule(mol)
                rdkit.MolToMolFile(mol, str(path.joinpath(f"{i}.mol")))
                csv_writer.writerow(column.to_list())
    # Median value selection.
    elif "Medium" in str(path):
        with open(str(path.joinpath("Scores.csv")), "w") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(df_unique_reaxys.columns)
            df_funcgroup = (
                df_unique_reaxys[df_unique_reaxys["func_group"] == func_group]
            )
            df_median = df_funcgroup[df_funcgroup[model_name] == df_funcgroup[model_name].median()].sort_values(model_name)
            df_above_median = df_funcgroup[df_funcgroup[model_name] > df_funcgroup[model_name].median()].sort_values(model_name)[0:4]
            df_below_median = df_funcgroup[df_funcgroup[model_name] < df_funcgroup[model_name].median()].sort_values(model_name, ascending=False)[0:4]
            df_combined = pd.concat([df_above_median, df_below_median, df_median])
            for i in range(len(df_combined)):
                column = df_combined.iloc[i]
                smiles = column["smiles"]
                mol = rdkit.MolFromSmiles(smiles)
                rdkit.EmbedMolecule(mol)
                rdkit.MolToMolFile(mol, str(path.joinpath(f"{i}.mol")))
                csv_writer.writerow(column.to_list())

In [5]:
# Loading Reaxys data
con = create_engine(
    "sqlite:///Data/Testing_Data/Reaxys_DB_Scored_AddHs.db"
)
df_reaxys = pd.read_sql(sql="synthetic_accesibility", con=con)
# Perform standardization on all molecules in Reaxys.
df_reaxys['standardized_smiles'] = [standardize_smiles(smi) for smi in df_reaxys['smiles']]

In [ ]:
con = create_engine(
    "sqlite:///Data/Testing_Data/Molder_DB_SA_AddHs.db"
)
df_training = pd.read_sql(sql='synthetic_accesibility', con=con)

In [ ]:
# Plotting score distributions for molecules in Reaxys.
fig, ax = plt.subplots()
sns.distplot(ax=ax, a=df_reaxys["sascore"], label="SAScore", kde=True, hist=False)
sns.distplot(ax=ax, a=df_reaxys["scscore"], label="SCScore", kde=True, hist=False)
sns.distplot(ax=ax, a=df_reaxys["rfmodel"], label="RFModel", kde=True, hist=False)
sns.despine()
ax.set_ylabel("Probability Density")
ax.set_xlabel("Synthetic Difficulty for Molecules in Reaxys Database / no units")

In [ ]:
fig, ax = plt.subplots()
sns.distplot(ax=ax, a=df_training["sascore"], label="SAScore", kde=True, hist=False)
sns.distplot(ax=ax, a=df_training["scscore"], label="SCScore", kde=True, hist=False)
sns.distplot(ax=ax, a=df_training["rfmodel"], label="RFModel", kde=True, hist=False)
sns.despine()
ax.set_ylabel("Probability Density")
ax.set_xlabel("Synthetic Difficulty for Molecules in Training Set / no units")

In [ ]:
# Database from the old EA paper.
reaxys_full = Path(
    "/Users/stevenbennett/Desktop/ga_paper/clean"
)
reaxys_full_amines = list(reaxys_full.glob("amines2f/*.mol"))
reaxys_full_aldehydes = list(reaxys_full.glob("aldehydes3f/*.mol"))

In [ ]:
# Plots the graph for SAScore vs SCScore with different functional groups.
palette = it.cycle(sns.color_palette('colorblind'))
fig, ax = plt.subplots()
sns.scatterplot(x=df_reaxys[df_reaxys['func_group'] == 'primary amine']['scscore'], y=df_reaxys[df_reaxys['func_group'] == 'primary amine']['sascore'], ax=ax, s=10, edgecolor='black', linewidth=0.4, color=next(palette), label='Di-topic amines')
sns.scatterplot(x=df_reaxys[df_reaxys['func_group'] == 'aldehyde']['scscore'], y=df_reaxys[df_reaxys['func_group'] == 'aldehyde']['sascore'], ax=ax, s=10, edgecolor='black', linewidth=0.4, color=next(palette), label='Tri-topic aldehydes')
ax.set_ylabel('SAScore')
ax.set_xlabel('SCScore')
sns.despine()
plt.figure(dpi=1200)
# fig.savefig(figure_path / 'Reaxys_DB_SA_SCScores_FuncGroups.pdf')


In [ ]:
# Plots the graph for SAScore vs SCScore with seperation by ML prediction.
palette = sns.color_palette('colorblind')
fig, ax = plt.subplots()
sns.scatterplot(x=df_reaxys[df_reaxys['rfmodel'] < 0.5]['scscore'], color=palette[2], y=df_reaxys[df_reaxys['rfmodel'] < 0.5]['sascore'], ax=ax, s=8, edgecolor='black', linewidth=0.7, label='Synthesisable')
sns.scatterplot(x=df_reaxys[df_reaxys['rfmodel'] > 0.5]['scscore'], color=palette[3], y=df_reaxys[df_reaxys['rfmodel'] > 0.5]['sascore'], ax=ax, s=8, edgecolor='black', linewidth=0.7, label='Unsynthesisable')
ax.set_ylabel('SAScore')
ax.set_xlabel('SCScore')
sns.despine()
plt.figure(dpi=1200)
fig.savefig(figure_path / 'Reaxys_DB_SA_SCScores_SynthesisableUnsythesisable.pdf')

In [ ]:
# Loads cages with lowest synthetic accesibility scores.
optimised_cages_path = (
    Path.home()
    / "PhD/main_projects/synthetic_accessibility_project/stages/optimisation_run_0"
)
optimised_cages = [
    stk.ConstructedMolecule.load(str(p)) for p in tqdm(optimised_cages_path.glob("*/*.json"))
]

In [ ]:
opt_cages_dict = pd.DataFrame({
    'BB1': [rdkit.MolToInchi(bb.to_rdkit_mol()) for cage in tqdm(optimised_cages) for bb in it.islice(cage.get_building_blocks(), 0, 1, 1)],
    'BB2': [rdkit.MolToInchi(bb.to_rdkit_mol()) for cage in tqdm(optimised_cages) for bb in it.islice(cage.get_building_blocks(), 1, 2, 1)],
    'SMILES': [rdkit.MolToInchi(cage.to_rdkit_mol()) for cage in tqdm(optimised_cages)]
})

In [ ]:
combinations = it.product(
    (r[1] for r in df_reaxys[df_reaxys["func_group"] == "primary amine"].iterrows()),
    (r[1] for r in df_reaxys[df_reaxys["func_group"] == "aldehyde"].iterrows()),
)
dict_combinations = defaultdict(list)

for combination in tqdm(combinations):
    dict_combinations["BB"].append(
        rdkit.MolToInchi(rdkit.MolFromSmiles(combination[0]["smiles"]))
        + " "
        + rdkit.MolToInchi(rdkit.MolFromSmiles(combination[1]["smiles"]))
    )
    dict_combinations["SAScore"].append(sum(r["sascore"] for r in combination))
    dict_combinations["SCScore"].append(sum(r["scscore"] for r in combination))
    dict_combinations["RFModel"].append(sum(r["rfmodel"] for r in combination))
df_combinations = pd.DataFrame(dict_combinations)

In [ ]:
df_sascore_onepercent = df_combinations[df_combinations['SAScore'] < df_combinations.quantile(.01)['SAScore']]
df_scscore_onepercent = df_combinations[df_combinations['SCScore'] < df_combinations.quantile(.01)['SCScore']]
df_rfmodel_onepercent = df_combinations[df_combinations['RFModel'] < df_combinations.quantile(.01)['RFModel']]

In [ ]:
list(it.islice(
        df_reaxys[df_reaxys["func_group"] == "primary amine"].iterrows(), 0, 1, 1
    ))

In [ ]:
df_opt_cages['BB InChI'] = [r[1]['BB2'] + ' ' + r[1]['BB1'] for r in df_opt_cages.iterrows()]

In [ ]:
df_scscore_onepercent[df_scscore_onepercent['BB SMILES'].isin(df_opt_cages['BB SMILES'])]

In [ ]:
df_reaxys.iloc[0]['smiles']

In [ ]:
df_opt_cages.iloc[0]['BB1'] 

In [ ]:
df_combinations

In [ ]:
df_reaxys['InChI'] = [rdkit.MolToInchi(rdkit.MolFromSmiles(r[1]['smiles'])) for r in df_reaxys.iterrows()]

In [ ]:
df_combinations['BB1'] = [r[1]['BB'].split(' ')[0] for r in tqdm(df_combinations.iterrows())]
df_combinations['BB2'] = [r[1]['BB'].split(' ')[1] for r in tqdm(df_combinations.iterrows())]

In [ ]:
df_opt_cages[df_opt_cages['BB2'].isin(df_reaxys['InChI'])]

In [ ]:
len(df_opt_cages[df_opt_cages['BB InChI'].isin(df_scscore_onepercent['BB'])])

In [ ]:
from pybel import *

In [ ]:
readstring('smiles', 'CCCC')

In [ ]:
from openbabel import pybel

In [ ]:
df_reaxys

In [ ]:
df_reaxys['InChI OBabel'] = [pybel.readstring('smi', r[1]['smiles']).write('inchi') for r in df_reaxys.iterrows()]

In [ ]:
for r in df_reaxys.iterrows():
    if r[1]['InChI'] != r[1]['InChI OBabel']:
        print(r[1]['InChI'])
        print(r[1]['InChI OBabel'])